In [11]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import pickle 

In [25]:
capacity = 1000
filename = 'alcohol_test'
data = pd.read_pickle(f'../data/data_pickle/{filename}.pkl')
if len(data) > 1000:
    data = data.head(capacity)
print(data)
data['url'] = data['severity_rating']
data['None'] = None
data['Mild'] = None
data['Moderate'] = None
data['Severe'] = None
data = data.drop('severity_rating', axis=1)
data

       IMDb_id                                    severity_rating
0    tt0057590  https://www.imdb.com/title/tt0057590/parentalg...
1    tt4701724  https://www.imdb.com/title/tt4701724/parentalg...
2    tt0163988  https://www.imdb.com/title/tt0163988/parentalg...
3    tt2308733  https://www.imdb.com/title/tt2308733/parentalg...
4    tt0362269  https://www.imdb.com/title/tt0362269/parentalg...
..         ...                                                ...
438  tt0025878  https://www.imdb.com/title/tt0025878/parentalg...
439  tt2452386  https://www.imdb.com/title/tt2452386/parentalg...
440  tt0268126  https://www.imdb.com/title/tt0268126/parentalg...
441  tt4834220  https://www.imdb.com/title/tt4834220/parentalg...
442  tt0053285  https://www.imdb.com/title/tt0053285/parentalg...

[443 rows x 2 columns]


,IMDb_id,url,None,Mild,Moderate,Severe
0,tt0057590,https://www.imdb.com/title/tt0057590/parentalg...,None,None,None,None
1,tt4701724,https://www.imdb.com/title/tt4701724/parentalg...,None,None,None,None
2,tt0163988,https://www.imdb.com/title/tt0163988/parentalg...,None,None,None,None
3,tt2308733,https://www.imdb.com/title/tt2308733/parentalg...,None,None,None,None
4,tt0362269,https://www.imdb.com/title/tt0362269/parentalg...,None,None,None,None
...,...,...,...,...,...,...
438,tt0025878,https://www.imdb.com/title/tt0025878/parentalg...,None,None,None,None
439,tt2452386,https://www.imdb.com/title/tt2452386/parentalg...,None,None,None,None
440,tt0268126,https://www.imdb.com/title/tt0268126/parentalg...,None,None,None,None
441,tt4834220,https://www.imdb.com/title/tt4834220/parentalg...,None,None,None,None


In [26]:
number = 0
for imdb, url in zip(data['IMDb_id'], data['url'], ):
    number += 1
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Gdy chcemy pobrać jakąś z cech to sprawdzamy jak na stronie to się nazywa najczęściej ma to format advisory-{emocja-rzecz} np: advisory-violence
    section = soup.find(id='advisory-nudity')
    # Znalezienie wszystkich <span> o klasie 'ipl-vote-button__details' w tej sekcji
    spans = section.find_all('span', class_='ipl-vote-button__details')
    if len(spans) >= 4:
        data['None'] = np.where(data['IMDb_id'] == imdb, spans[0].text, data['None'])
        data['Mild'] = np.where(data['IMDb_id'] == imdb, spans[1].text, data['Mild'])
        data['Moderate'] = np.where(data['IMDb_id'] == imdb, spans[2].text, data['Moderate'])
        data['Severe'] = np.where(data['IMDb_id'] == imdb, spans[3].text, data['Severe'])
    if number == capacity:
        break
print("Skończone pobieranie")

Skończone pobieranie


In [27]:
data

,IMDb_id,url,None,Mild,Moderate,Severe
0,tt0057590,https://www.imdb.com/title/tt0057590/parentalg...,7,8,5,0
1,tt4701724,https://www.imdb.com/title/tt4701724/parentalg...,29,11,1,5
2,tt0163988,https://www.imdb.com/title/tt0163988/parentalg...,25,17,4,1
3,tt2308733,https://www.imdb.com/title/tt2308733/parentalg...,15,6,1,10
4,tt0362269,https://www.imdb.com/title/tt0362269/parentalg...,3,2,6,24
...,...,...,...,...,...,...
438,tt0025878,https://www.imdb.com/title/tt0025878/parentalg...,12,2,0,0
439,tt2452386,https://www.imdb.com/title/tt2452386/parentalg...,22,31,6,1
440,tt0268126,https://www.imdb.com/title/tt0268126/parentalg...,7,14,31,11
441,tt4834220,https://www.imdb.com/title/tt4834220/parentalg...,3,2,2,0


In [28]:
# Zapisanie do pikla
with open(f"../notebooks/training_pkl/{filename}{capacity}.pkl", "wb") as f:
    pickle.dump(data, f)

In [29]:
data = pd.read_pickle(f"training_pkl/{filename}{capacity}.pkl")
data

,IMDb_id,url,None,Mild,Moderate,Severe
0,tt0057590,https://www.imdb.com/title/tt0057590/parentalg...,7,8,5,0
1,tt4701724,https://www.imdb.com/title/tt4701724/parentalg...,29,11,1,5
2,tt0163988,https://www.imdb.com/title/tt0163988/parentalg...,25,17,4,1
3,tt2308733,https://www.imdb.com/title/tt2308733/parentalg...,15,6,1,10
4,tt0362269,https://www.imdb.com/title/tt0362269/parentalg...,3,2,6,24
...,...,...,...,...,...,...
438,tt0025878,https://www.imdb.com/title/tt0025878/parentalg...,12,2,0,0
439,tt2452386,https://www.imdb.com/title/tt2452386/parentalg...,22,31,6,1
440,tt0268126,https://www.imdb.com/title/tt0268126/parentalg...,7,14,31,11
441,tt4834220,https://www.imdb.com/title/tt4834220/parentalg...,3,2,2,0
